# Backpropagation Practice

Implement a 3 input, 4 node hidden-layer, 1 output node Multilayer Perceptron on the following dataset:

| x1 | x2 | x3 | y |
|----|----|----|---|
| 0  | 0  | 1  | 0 |
| 0  | 1  | 1  | 1 |
| 1  | 0  | 1  | 1 |
| 0  | 1  | 0  | 1 |
| 1  | 0  | 0  | 1 |
| 1  | 1  | 1  | 0 |
| 0  | 0  | 0  | 0 |

If you look at the data you'll notice that the first two columns behave like an XOR gate while the last column is mostly just noise. Remember that creating an XOR gate was what the perceptron was criticized for not being able to learn. 

In [1]:
import numpy as np

X = np.array((
    [0,0,1],
    [0,1,1],
    [1,0,1],
    [0,1,0],
    [1,0,0],
    [1,1,1],
    [0,0,0]
             ), dtype=float)

y = np.array((
    [0],
    [1],
    [1],
    [1],
    [1],
    [0],
    [0]
        ), dtype=float)


In [7]:
class NeuralNetwork:
    def __init__(self):
        # 3 input, 4 hidden, 1 output layers
        self.input = 3
        self.hiddenNodes = 4
        self.outputNodes = 1
        
        # initial weights
        self.weights1 = np.random.randn(self.input, self.hiddenNodes)
        
        self.weights2 = np.random.randn(self.hiddenNodes, self.outputNodes)
    
    def sigmoid(self, s):
        return 1 / (1 + np.exp(-s))
    
    def sigmoidPrime(self, s):
        return s * (1 - s)
    
    def feed_forward(self, X):
        # weighted sums of inputs and hidden
        self.hidden_sum = np.dot(X, self.weights1)
        
        #activations of weighted sums
        self.activated_hidden = self.sigmoid(self.hidden_sum)
        
        # weighted sum between hidden and output
        self.output_sum = np.dot(self.activated_hidden, self.weights2)
        
        # final activation of output
        self.activated_output = self.sigmoid(self.output_sum)
        
        return self.activated_output
    
    def backward(self, X, y, o):
        
        self.o_error = y - o # error in output
        self.o_delta = self.o_error * self.sigmoidPrime(o) # apply derivative of sigmoid error
        
        self.z2_error = self.o_delta.dot(self.weights2.T) # z2 error: how much our hidden layer weights were off
        self.z2_delta = self.z2_error * self.sigmoidPrime(self.activated_hidden)
        
        self.weights1 += X.T.dot(self.z2_delta) # adjust first set (input => hidden) weights
        self.weights2 += self.activated_hidden.T.dot(self.o_delta) # adjust second set (hidden => output) weights
        
    def train(self, X, y):
        o = self.feed_forward(X)
        self.backward(X, y, o)        

In [8]:
nn = NeuralNetwork()

for epoch in range(1000):
    if (epoch+1 in [1,2,3,4,5]) or ((epoch+1) % 50 == 0):
        print('+' + '---' * 3 + f'EPOCH {epoch+1}' + '---'*3 + '+')
        print('Input: \n', X)
        print('Actual Output: \n', y)
        print('Predicted Output: \n', str(nn.feed_forward(X)))
        print('Loss: \n', str(np.mean(np.square(y - nn.feed_forward(X)))))
    nn.train(X, y)

+---------EPOCH 1---------+
Input: 
 [[0. 0. 1.]
 [0. 1. 1.]
 [1. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 1. 1.]
 [0. 0. 0.]]
Actual Output: 
 [[0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]]
Predicted Output: 
 [[0.88117773]
 [0.858956  ]
 [0.9187312 ]
 [0.88000972]
 [0.91630369]
 [0.91219693]
 [0.90348534]]
Loss: 
 0.35325199362556253
+---------EPOCH 2---------+
Input: 
 [[0. 0. 1.]
 [0. 1. 1.]
 [1. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 1. 1.]
 [0. 0. 0.]]
Actual Output: 
 [[0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]]
Predicted Output: 
 [[0.84798049]
 [0.82404849]
 [0.88848115]
 [0.85661566]
 [0.89216014]
 [0.88069371]
 [0.88262282]]
Loss: 
 0.3356141808284945
+---------EPOCH 3---------+
Input: 
 [[0. 0. 1.]
 [0. 1. 1.]
 [1. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 1. 1.]
 [0. 0. 0.]]
Actual Output: 
 [[0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]]
Predicted Output: 
 [[0.80250131]
 [0.77782935]
 [0.84340757]
 [0.82593188]
 [0.85784118]
 [0.83459624]
 [0.85440233]]
Loss: 
 0.31356463165365184
+---------EPOC

## Try building/training a more complex MLP on a bigger dataset.

Use the [MNIST dataset](http://yann.lecun.com/exdb/mnist/) to build the cannonical handwriting digit recognizer and see what kind of accuracy you can achieve. 

If you need inspiration, the internet is chalk-full of tutorials, but I want you to see how far you can get on your own first. I've linked to the original MNIST dataset above but it will probably be easier to download data through a neural network library. If you reference outside resources make sure you understand every line of code that you're using from other sources, and share with your fellow students helpful resources that you find.


In [ ]:
##### Your Code Here #####

## Stretch Goals: 

- Implement Cross Validation model evaluation on your MNIST implementation 
- Research different [Gradient Descent Based Optimizers](https://keras.io/optimizers/)
 - [Siraj Raval the evolution of gradient descent](https://www.youtube.com/watch?v=nhqo0u1a6fw)
- Build a housing price estimation model using a neural network. How does its accuracy compare with the regression models that we fit earlier on in class?